In [ ]:
#
# Based on the EDx course Math 226.2 by Paul Blanchard
# Julia version derived by Rob J Goedman
# April 2016
#

using Plots
pyplot(reuse=true)
plot()

#
# y'[t] = -x[t]
# x'[t]= -3 x[t] + y[t]
#
# or
#
# y''[t] = -3 y'[t] + y[t]
#
# In matrix form:
#
m = [-3 1; -1 0]

e = eigfact(m)
e |> display
println()

#
# 2 negative eigenvalues: slow eigen line will act as a sink
#

function xf(k::Vector{Float64}, t::Float64)
  k[1] * e[:vectors][1, 1] * exp(e[:values][1] * t) +
    k[2] * e[:vectors][1, 2] * exp(e[:values][2] * t)
end

function yf(k::Vector{Float64}, t::Float64)
  k[1] * e[:vectors][2, 1] * exp(e[:values][1] * t) +
    k[2] * e[:vectors][2, 2] * exp(e[:values][2] * t)
end

t1 = -5.0:0.01:5.0
k = e[:vectors]\e[:vectors][:,1]
# Extend eigen lines to cover full range
xfast = [xf(-k, t) for t in t1]
yfast = [yf(-k, t) for t in t1]
xfast = [xfast; [xf(k, t) for t in t1]]
yfast = [yfast; [yf(k, t) for t in t1]]

t2 = -5.0:0.01:5.0
k = e[:vectors]\e[:vectors][:,2]
xslow = [xf(-k, t) for t in t2]
yslow = [yf(-k, t) for t in t2]
xslow = [xslow; [xf(k, t) for t in t2]]
yslow = [yslow; [yf(k, t) for t in t2]]

t3 = -0.15:0.01:5.0
k = e[:vectors]\[-3, 2]
xvals = [xf(k, t) for t in t3]
yvals = [yf(k, t) for t in t3]

N = 40
vals = collect(linspace(-6, 6, N))
x = vals'
y = vals
for i in 1:N-1
  x = vcat(x, vals')
  y = hcat(y, vals)
end

u = zeros(N, N)
u = m[1, 1]*x + m[1, 2]*y
v = zeros(N, N)
v = m[2, 1]*x + m[2, 2]*y

PyPlot.streamplot(x, y, u, v, color=u, density=0.7)
plot!(title = "Streamplot", xaxis=("x(t)", (-4,4)), yaxis=("y(t)", (-4,4)))
plot!(xvals, yvals, color=:black, linewidth=2, linestyle=:dot, lab="Y[0]=[-3,2] soln")
plot!(xfast, yfast, color=:red, linewidth=2, lab="Fast eigen line")
plot!(xslow, yslow, color=:green, linewidth=2, lab="Slow eigen line")

png("/Users/rob/Desktop/streamplot.png")